# Literature Review

A common task while exploring a new topic is to conduct a literature review. In this example we will explore how a multi-agent team can be configured to conduct a _simple_ literature review.

- **Arxiv Search Agent**: Use the Arxiv API to search for papers related to a given topic and return results.
- **Google Search Agent**: Use the Google Search api to find papers related to a given topic and return results.
- **Report Agent**: Generate a report based on the information collected by the search and stock analysis agents.


First, let us import the necessary modules. 

In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.teams.group_chat import RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

## Defining Tools 

Next, we will define the tools that the agents will use to perform their tasks. In this case we will define a simple function `search_arxiv` that will use the `arxiv` library to search for papers related to a given topic.  

Finally, we will wrap the functions into a `FunctionTool` class that will allow us to use it as a tool in the agents. 

Note: You will need to set the appropriate environment variables for tools as needed.

Also install required libraries: 

```bash
!pip install arxiv
```

In [2]:

def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import os
    import requests
    from dotenv import load_dotenv
    from bs4 import BeautifulSoup
    import time

    load_dotenv()

    api_key = os.getenv('GOOGLE_API_KEY')
    search_engine_id = os.getenv('GOOGLE_SEARCH_ENGINE_ID')
    
    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'key': api_key,
        'cx': search_engine_id,
        'q': query,
        'num': num_results
    }

    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get('items', [])
    
    def get_page_content(url):
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text(separator=' ', strip=True)
            words = text.split()
            content = ''
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += ' ' + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item['link'])
        enriched_results.append({
            'title': item['title'],
            'link': item['link'],
            'snippet': item['snippet'],
            'body': body
        })
        time.sleep(1)  # Be respectful to the servers

    return enriched_results

def arxiv_search(query: str, max_results: int = 2) -> list:
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv  
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    results = []
    for paper in client.results(search):
        results.append({
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'published': paper.published.strftime('%Y-%m-%d'),
            'abstract': paper.summary,
            'pdf_url': paper.pdf_url,
        })
    
    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)
    
    return results

In [3]:
google_search_tool = FunctionTool(google_search, description="Search Google for information, returns results with a snippet and body content")
arxiv_search_tool = FunctionTool(arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts")

## Defining Agents 

Next, we will define the agents that will perform the tasks. 

In [4]:
google_search_agent = ToolUseAssistantAgent(
    name="Google_Search_Agent",
    registered_tools=[google_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Google for information, returns results with a snippet and body content", 
    system_message  = "You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = ToolUseAssistantAgent(
    name="Arxiv_Search_Agent",
    registered_tools=[ arxiv_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts", 
    system_message  = "You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = CodingAssistantAgent( 
    name="Report_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a report based on a given topic",  
    system_message  = "You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)

team = RoundRobinGroupChat( 
    participants=[google_search_agent, arxiv_search_agent, report_agent]
)

result = await team.run(task="Write a literature review on no code tools for building multi agent ai systems")
result


--------------------------------------------------------------------------- 
[2024-10-08T10:21:08.443314]:

Write a literature review on no code tools for building multi agent ai systems
From: user
--------------------------------------------------------------------------- 
[2024-10-08T10:21:09.295153], Google_Search_Agent:

[FunctionCall(id='call_hw2vekTxYJgfPynNxomuQQhU', arguments='{"query":"no code tools for building multi agent AI systems","num_results":5}', name='google_search')]
From: Google_Search_Agent
--------------------------------------------------------------------------- 
[2024-10-08T10:21:16.023771], tool_agent_for_Google_Search_Agent:

[FunctionExecutionResult(content='[{\'title\': \'AutoGen Studio: A No-Code Developer Tool for Building and ...\', \'link\': \'https://arxiv.org/abs/2408.15247\', \'snippet\': \'Aug 9, 2024 ... Abstract:Multi-agent systems, where multiple agents (generative AI models + tools) collaborate, are emerging as an effective pattern for\\xa0...\

TeamRunResult(messages=[TextMessage(source='user', content='Write a literature review on no code tools for building multi agent ai systems'), TextMessage(source='Google_Search_Agent', content='### Literature Review on No-Code Tools for Building Multi-Agent AI Systems\n\nThe field of artificial intelligence is rapidly evolving, particularly with the emergence of multi-agent systems (MAS), where multiple intelligent agents collaborate to solve complex problems. The advent of no-code tools has significantly democratized access to these technologies, enabling non-technical users to construct and deploy multi-agent systems without extensive programming knowledge. This review synthesizes recent findings and developments in no-code tools suitable for building MAS.\n\n1. **AutoGen Studio**: Recent advancements have shown that tools like AutoGen Studio are at the forefront of the no-code revolution in multi-agent system development. According to a study published on arXiv, AutoGen Studio serves